In [1]:
import csv
import numpy as np
import os

goeievraag = '/roaming/fkunnema/goeievraag/exp_similarity'

In [2]:
# precision at N
def precision_N(y_real, y_pred, N):
    assert N > 0
    tp = len(list(filter(lambda y: y in y_real, y_pred[:N])))
    return float(tp) / N

# mean averaged precision
def MAP(y_real, y_pred, N):
    precisions = []
    for i in range(1, N):
        precisions.append(precision_N(y_real, y_pred, i))
    
    return np.mean(precisions)

In [3]:
# seed and unseed
with open(os.path.join(goeievraag, 'seed_questions.txt')) as f:
    _seed_questions = f.read().strip().split('\n')
with open(os.path.join(goeievraag, 'seed_questions.tok.txt')) as f:
    seed_questions = f.read().strip().split('\n')
with open(os.path.join(goeievraag, 'unseeded_questions.txt')) as f:
    _unseed_questions = f.read().strip().split('\n')
with open(os.path.join(goeievraag, 'unseeded_questions.tok.txt')) as f:
    unseed_questions = f.read().strip().split('\n')

In [4]:
seed_questions[0]

'Waar vind ik een stopwatch met in te stellen alarm bv om de 3 minuten en displayverlichting'

In [9]:
# y_real
real = []
path = 'rankings/ranked_questions_annotated_FK_TCF_combined.csv'
with open(path) as f:
    reader = csv.reader(f)
    for i, row in enumerate(reader):
        if i > 0:
            print('Element: ', i, end='\t\r')
            seed, retrieval, score, florian, thiago, _, combined = row
            y = {
                'seed': seed,
                'retrieval': retrieval,
                'score': score,
                'florian': florian,
                'thiago': thiago,
                'combined': combined
            }
            
            y['seed_tok'] = ''
            for j, candidate in enumerate(_seed_questions):
                if seed.lower() == candidate.lower():
                    y['seed_tok'] = seed_questions[j]
                    break
                    
            y['retrieval_tok'] = ''
            for j, candidate in enumerate(_unseed_questions):
                if retrieval.lower() == candidate.lower():
                    y['retrieval_tok'] = unseed_questions[j]
                    break
                    
            real.append(y)

y_seed, n_seed = 0,0
y_unseed, n_unseed = 0,0
for question in real:
    if question['seed_tok'] != '':
        y_seed += 1
    else:
        n_seed += 1
        
    if question['retrieval_tok'] != '':
        y_unseed += 1
    else:
        n_unseed += 1

print('\n')
print('Seed Found:', y_seed)
print('Seed Not Found:', n_seed)

print('Retrieval Found:', y_unseed)
print('Retrieval Not Found:', n_unseed)

Element:  1100	40	 73	 102	 113	115	 149	157	 194	 227		273	 294	 313	 319		 346		358		 366	 372	 380		 407	411	 429	 440	 444	446	 453		 483	497	 512	 516	524	534	 555	584	606	 609	633	 635	 638	667	 674	 717		737	 755	763	 765		 786		796	 800		 821		886	 888	 900	915	 945	975	 988	 996	 999			 1039	 1041			

Seed Found: 1089
Seed Not Found: 11
Retrieval Found: 1086
Retrieval Not Found: 14


In [14]:
# save gold ranking
path = 'rankings/ranking.tok.csv'
with open(path, 'w') as f:
    writer = csv.writer(f, delimiter=',', quotechar='\'', quoting=csv.QUOTE_MINIMAL)
    for row in real:
        writer.writerow([row['seed_tok'].lower(), row['retrieval_tok'].lower(), row['score'], row['florian'], row['thiago'], '', row['combined']])

In [14]:
# Language Model, Translation Model, Translation-based Language Model
lm = []
path = 'rankings/lmranking.csv'
with open(path) as f:
    reader = csv.reader(f)
    for row in reader:
        seed_tok, retrieval_tok, score = row
        lm.append({
            'seed_tok': seed_tok,
            'retrieval_tok': retrieval_tok,
            'score': float(score)
        })
        
trm = []
path = 'rankings/trmranking.csv'
with open(path) as f:
    reader = csv.reader(f)
    for row in reader:
        seed_tok, retrieval_tok, score = row
        trm.append({
            'seed_tok': seed_tok,
            'retrieval_tok': retrieval_tok,
            'score': float(score)
        })
        
trlm = []
path = 'rankings/trlmranking.csv'
with open(path) as f:
    reader = csv.reader(f)
    for row in reader:
        seed_tok, retrieval_tok, score = row
        trlm.append({
            'seed_tok': seed_tok,
            'retrieval_tok': retrieval_tok,
            'score': float(score)
        })

In [24]:
# Retrieve gold-standards according to Florian
print('Instances: ', len(real))
thiago = list(filter(lambda x: x['thiago']=='1', real))
print('Relevant for Thiago: ', len(thiago))
florian = list(filter(lambda x: x['florian']=='1', real))
print('Relevant for Florian: ', len(florian))
gold = list(filter(lambda x: x['combined']=='1', real))
print('Relevant for Both: ', len(gold))

seeds = list(set(map(lambda x: x['seed_tok'], gold)))
print('Relevant for Seeds: ', len(seeds))

Instances:  1100
Relevant for Thiago:  222
Relevant for Florian:  76
Relevant for Both:  60
Relevant for Seeds:  29


In [59]:
# lm evaluation
for seed in seeds[6:7]:
    f = filter(lambda x: x['seed_tok'].lower() == seed.lower(), gold)
    y_real = list(map(lambda x: x['retrieval_tok'].lower(), f))
    print(len(y_real))
    
    print('REAL')
    for y in y_real:
        print(y)
    
    f = filter(lambda x: x['seed_tok'].lower() == seed.lower(), lm)
    f = sorted(map(lambda x: (x['retrieval_tok'].lower(), float(x['score'])), f), key=lambda x:x[1], reverse=True)
    y_pred = list(map(lambda x: x[0], f))
#     print('LM Precision:', precision_N(y_real, y_pred, 10))
    
    f = filter(lambda x: x['seed_tok'].lower() == seed.lower(), trm)
    f = sorted(map(lambda x: (x['retrieval_tok'].lower(), float(x['score'])), f), key=lambda x:x[1], reverse=True)
    y_pred = list(map(lambda x: x[0], f))
#     print('TRM Precision:', precision_N(y_real, y_pred, 10))
    
    f = filter(lambda x: x['seed_tok'].lower() == seed.lower(), trlm)
    f = sorted(map(lambda x: (x['retrieval_tok'].lower(), float(x['score'])), f), key=lambda x:x[1], reverse=True)
    y_pred = list(map(lambda x: x[0], f))
    print('TRLM Precision:', precision_N(y_real, y_pred, 10))
    print(10 * '-')
    

5
REAL
iphone 4s batterij enorm snel leeg
batterij iphone gaat snel leeg na aankoop action lader hoe kan dit
batterij iphone 5s gaat snel leeg is dit een normaal batterij gebruik
klopt dat door ios 6 je batterij sneller leeg gaat
is het normaal dat de accu van mijn telefoon zo snel leeg is
LM Precision: 0.0
TRM Precision: 0.0
TRLM Precision: 0.0
----------


['wat houdt een behandeling met radioactief jodium in bij een kat met acute schildklierproblemen', 'bult verdikking onder mijn ribbenkast wat zou het kunnen zijn', 'waarom doen sommige mensen hun hand onder de kin als ze slapen', 'hoe weet je of je kat een tumor heeft inplaats van een vet bult', 'mijn kat heeft een bult op zijn rug een harde bult hij eet bijna niet meer en drinkt ook bijna niet meer ????????', 'hoe kom ik af van die rare bult onder mijn rug', 'ik heb een bult onder rechteroksel die er al een tijdje zit wat kan het zijn', 'wat is die bult onder tanden hij is pijnlijk en hard', 'waarom heeft een geit aan weerszijden van zijn mond onder zijn kin een zakje hangen hoe heten die zakjes', 'waarom ontstaat er een witte bult onder een blarenpleister']


['wat kan een bult op een kattenlip zijn']